In [1]:
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
import warnings
%pylab inline
warnings.filterwarnings('ignore')


Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv(r'C:\Users\admcc\Desktop\aac_shelter_outcomes.csv')
df.columns.values

array(['age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
       'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype',
       'outcome_type', 'sex_upon_outcome'], dtype=object)

In [3]:
df.drop_duplicates(['animal_id'], keep='last')

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown
5,4 months,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male
6,1 year,A693700,Other,Squirrel Mix,Tan,2013-12-13T00:00:00,2014-12-13T12:20:00,2014-12-13T12:20:00,NaN,Suffering,Euthanasia,Unknown
7,3 years,A692618,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23T00:00:00,2014-12-08T15:55:00,2014-12-08T15:55:00,*Ella,Partner,Transfer,Spayed Female
8,1 month,A685067,Cat,Domestic Shorthair Mix,Blue Tabby/White,2014-06-16T00:00:00,2014-08-14T18:45:00,2014-08-14T18:45:00,Lucy,NaN,Adoption,Intact Female
9,3 months,A678580,Cat,Domestic Shorthair Mix,White/Black,2014-03-26T00:00:00,2014-06-29T17:45:00,2014-06-29T17:45:00,*Frida,Offsite,Adoption,Spayed Female


In [4]:
df['breed'].value_counts

<bound method Series.value_counts of 0                      Domestic Shorthair Mix
1                                  Beagle Mix
2                                    Pit Bull
3                     Miniature Schnauzer Mix
4                                     Bat Mix
5                              Leonberger Mix
6                                Squirrel Mix
7                     Chihuahua Shorthair Mix
8                      Domestic Shorthair Mix
9                      Domestic Shorthair Mix
10                   Domestic Medium Hair Mix
11                     Papillon/Border Collie
12             Chihuahua Shorthair/Pomeranian
13       Miniature Schnauzer/Miniature Poodle
14                     Labrador Retriever Mix
15                            Rat Terrier Mix
16                               Pit Bull Mix
17                     Domestic Shorthair Mix
18         German Shepherd/Labrador Retriever
19                     Domestic Shorthair Mix
20                     Domestic Shorthair M

In [5]:
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], format='%Y-%m-%d')
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d')

df['days_alive'] = df['datetime'].sub(df['date_of_birth'], axis=0)
df['days_alive'] = df['days_alive'].dt.days
df.describe()

,days_alive
count,78256.000000
mean,817.034822
std,1077.098289
min,-123.000000
25%,95.000000
50%,371.000000
75%,1096.000000
max,9137.000000


In [6]:
mask = df['name'].isnull()
df['has_name'] = 1
df['has_name'][mask] = 0

df['target'] = 0.0
mask = (df['outcome_type'] == 'Adoption')
df['target'][mask] = 1.0
df['target'].value_counts()

0.0    45144
1.0    33112
Name: target, dtype: int64

In [7]:

df['age_binned'] = pd.qcut(df['days_alive'], 5)
print df['age_binned'][:5]

0    (-123.001, 78.0]
1      (306.0, 669.0]
2      (306.0, 669.0]
3    (1222.0, 9137.0]
4       (78.0, 306.0]
Name: age_binned, dtype: category
Categories (5, interval[float64]): [(-123.001, 78.0] < (78.0, 306.0] < (306.0, 669.0] < (669.0, 1222.0] < (1222.0, 9137.0]]


In [8]:
def get_month(date):
    return date.month

In [9]:
df['month'] = df['datetime'].map(get_month)
df

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,days_alive,has_name,target,age_binned,month
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22 16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male,15,0,0.0,"(-123.001, 78.0]",7
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06,2013-11-07 11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female,366,1,0.0,"(306.0, 669.0]",11
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31,2014-06-03 14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male,429,1,1.0,"(306.0, 669.0]",6
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15 15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male,3300,1,0.0,"(1222.0, 9137.0]",6
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07,2014-07-07 14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown,181,0,0.0,"(78.0, 306.0]",7
5,4 months,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03,2013-10-07 13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male,126,1,0.0,"(78.0, 306.0]",10
6,1 year,A693700,Other,Squirrel Mix,Tan,2013-12-13,2014-12-13 12:20:00,2014-12-13T12:20:00,NaN,Suffering,Euthanasia,Unknown,365,0,0.0,"(306.0, 669.0]",12
7,3 years,A692618,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23,2014-12-08 15:55:00,2014-12-08T15:55:00,*Ella,Partner,Transfer,Spayed Female,1111,1,0.0,"(669.0, 1222.0]",12
8,1 month,A685067,Cat,Domestic Shorthair Mix,Blue Tabby/White,2014-06-16,2014-08-14 18:45:00,2014-08-14T18:45:00,Lucy,NaN,Adoption,Intact Female,59,1,1.0,"(-123.001, 78.0]",8
9,3 months,A678580,Cat,Domestic Shorthair Mix,White/Black,2014-03-26,2014-06-29 17:45:00,2014-06-29T17:45:00,*Frida,Offsite,Adoption,Spayed Female,95,1,1.0,"(78.0, 306.0]",6


In [10]:
categorical_columns = ['age_binned', 'animal_type', 'sex_upon_outcome', 'month']
df_dummies = pd.get_dummies(df[categorical_columns],
                            prefix=categorical_columns,
                            columns=categorical_columns)
dummy_column_names = df_dummies.columns.values
dummy_column_names[:10]

array(['age_binned_(-123.001, 78.0]', 'age_binned_(78.0, 306.0]',
       'age_binned_(306.0, 669.0]', 'age_binned_(669.0, 1222.0]',
       'age_binned_(1222.0, 9137.0]', 'animal_type_Bird',
       'animal_type_Cat', 'animal_type_Dog', 'animal_type_Livestock',
       'animal_type_Other'], dtype=object)

In [11]:
df2 = pd.concat([df, df_dummies], axis=1)

In [12]:
formula = 'target ~ 0 + {}'.format(' + '.join(['Q("{}")'.format(x) for x in dummy_column_names]))
print formula

target ~ 0 + Q("age_binned_(-123.001, 78.0]") + Q("age_binned_(78.0, 306.0]") + Q("age_binned_(306.0, 669.0]") + Q("age_binned_(669.0, 1222.0]") + Q("age_binned_(1222.0, 9137.0]") + Q("animal_type_Bird") + Q("animal_type_Cat") + Q("animal_type_Dog") + Q("animal_type_Livestock") + Q("animal_type_Other") + Q("sex_upon_outcome_Intact Female") + Q("sex_upon_outcome_Intact Male") + Q("sex_upon_outcome_Neutered Male") + Q("sex_upon_outcome_Spayed Female") + Q("sex_upon_outcome_Unknown") + Q("month_1") + Q("month_2") + Q("month_3") + Q("month_4") + Q("month_5") + Q("month_6") + Q("month_7") + Q("month_8") + Q("month_9") + Q("month_10") + Q("month_11") + Q("month_12")


In [13]:
Y, X = dmatrices(formula, df2, return_type='dataframe')

In [14]:
y = Y['target'].values
y[:10]

array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.])

In [15]:
from sklearn import naive_bayes
model = naive_bayes.MultinomialNB()

In [16]:
model.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
print 'Prediction'
print model.predict(X[:10])
print 'Actual'
print y[:10]

Prediction
[ 0.  1.  1.  0.  0.  0.  0.  1.  0.  1.]
Actual
[ 0.  0.  1.  0.  0.  0.  0.  0.  1.  1.]


In [18]:
from sklearn import metrics

prediction_train = model.predict(X)
print metrics.accuracy_score(y, prediction_train)

0.733541198119


In [19]:
feature_importances = abs(model.feature_log_prob_[1] - model.feature_log_prob_[0])
feature_importances

array([ 0.0758732 ,  0.77161914,  0.00962255,  0.25524527,  0.46935594,
        0.34333434,  0.03922363,  0.12229434,  0.67093302,  2.6323164 ,
        2.08383113,  2.31533394,  0.60188336,  0.81664009,  4.207904  ,
        0.17017864,  0.10764613,  0.29755351,  0.39892568,  0.27070499,
        0.01012738,  0.27627736,  0.12816415,  0.04496438,  0.09878535,
        0.0344904 ,  0.25326377])

In [20]:
feature_importance_series = Series(feature_importances, index=X.columns.values)
feature_importance_series[:50]

Q("age_binned_(-123.001, 78.0]")       0.075873
Q("age_binned_(78.0, 306.0]")          0.771619
Q("age_binned_(306.0, 669.0]")         0.009623
Q("age_binned_(669.0, 1222.0]")        0.255245
Q("age_binned_(1222.0, 9137.0]")       0.469356
Q("animal_type_Bird")                  0.343334
Q("animal_type_Cat")                   0.039224
Q("animal_type_Dog")                   0.122294
Q("animal_type_Livestock")             0.670933
Q("animal_type_Other")                 2.632316
Q("sex_upon_outcome_Intact Female")    2.083831
Q("sex_upon_outcome_Intact Male")      2.315334
Q("sex_upon_outcome_Neutered Male")    0.601883
Q("sex_upon_outcome_Spayed Female")    0.816640
Q("sex_upon_outcome_Unknown")          4.207904
Q("month_1")                           0.170179
Q("month_2")                           0.107646
Q("month_3")                           0.297554
Q("month_4")                           0.398926
Q("month_5")                           0.270705
Q("month_6")                           0

In [21]:
feature_importance_series.sort_values(ascending=False)[:50]

Q("sex_upon_outcome_Unknown")          4.207904
Q("animal_type_Other")                 2.632316
Q("sex_upon_outcome_Intact Male")      2.315334
Q("sex_upon_outcome_Intact Female")    2.083831
Q("sex_upon_outcome_Spayed Female")    0.816640
Q("age_binned_(78.0, 306.0]")          0.771619
Q("animal_type_Livestock")             0.670933
Q("sex_upon_outcome_Neutered Male")    0.601883
Q("age_binned_(1222.0, 9137.0]")       0.469356
Q("month_4")                           0.398926
Q("animal_type_Bird")                  0.343334
Q("month_3")                           0.297554
Q("month_7")                           0.276277
Q("month_5")                           0.270705
Q("age_binned_(669.0, 1222.0]")        0.255245
Q("month_12")                          0.253264
Q("month_1")                           0.170179
Q("month_8")                           0.128164
Q("animal_type_Dog")                   0.122294
Q("month_2")                           0.107646
Q("month_10")                          0

In [22]:
top_10_feature_indices = feature_importance_series.sort_values(ascending=False)[:50].index.values

In [23]:
inter_class_differences = model.feature_log_prob_[1] - model.feature_log_prob_[0]
new_feature_importance_series = Series(inter_class_differences, index=X.columns.values)

new_feature_importance_series[top_10_feature_indices]

Q("sex_upon_outcome_Unknown")         -4.207904
Q("animal_type_Other")                -2.632316
Q("sex_upon_outcome_Intact Male")     -2.315334
Q("sex_upon_outcome_Intact Female")   -2.083831
Q("sex_upon_outcome_Spayed Female")    0.816640
Q("age_binned_(78.0, 306.0]")          0.771619
Q("animal_type_Livestock")            -0.670933
Q("sex_upon_outcome_Neutered Male")    0.601883
Q("age_binned_(1222.0, 9137.0]")      -0.469356
Q("month_4")                          -0.398926
Q("animal_type_Bird")                 -0.343334
Q("month_3")                          -0.297554
Q("month_7")                           0.276277
Q("month_5")                          -0.270705
Q("age_binned_(669.0, 1222.0]")       -0.255245
Q("month_12")                          0.253264
Q("month_1")                           0.170179
Q("month_8")                           0.128164
Q("animal_type_Dog")                   0.122294
Q("month_2")                           0.107646
Q("month_10")                         -0

In [24]:
import sys
sys.setrecursionlimit(10000)

In [25]:
df_copy = df.copy()
mask = df_copy['animal_type'].isin(["Dog", "Cat"])
df_copy = df_copy[mask]
df_copy
df_copy['target'] = 0.0
mask = (df['outcome_type'] == 'Adoption')
df_copy['target'][mask] = 1.0
df_copy['target'].value_counts()

0.0    40880
1.0    32784
Name: target, dtype: int64

In [26]:
categorical_columns = ['age_binned', 'sex_upon_outcome', 'month']
df_dummies = pd.get_dummies(df[categorical_columns],
                            prefix=categorical_columns,
                            columns=categorical_columns)
dummy_column_names = df_dummies.columns.values
dummy_column_names[:10]

array(['age_binned_(-123.001, 78.0]', 'age_binned_(78.0, 306.0]',
       'age_binned_(306.0, 669.0]', 'age_binned_(669.0, 1222.0]',
       'age_binned_(1222.0, 9137.0]', 'sex_upon_outcome_Intact Female',
       'sex_upon_outcome_Intact Male', 'sex_upon_outcome_Neutered Male',
       'sex_upon_outcome_Spayed Female', 'sex_upon_outcome_Unknown'], dtype=object)

In [27]:
df3 = pd.concat([df_copy, df_dummies], axis=1)

In [28]:
formula = 'target ~ 0 + {}'.format(' + '.join(['Q("{}")'.format(x) for x in dummy_column_names]))
print formula

target ~ 0 + Q("age_binned_(-123.001, 78.0]") + Q("age_binned_(78.0, 306.0]") + Q("age_binned_(306.0, 669.0]") + Q("age_binned_(669.0, 1222.0]") + Q("age_binned_(1222.0, 9137.0]") + Q("sex_upon_outcome_Intact Female") + Q("sex_upon_outcome_Intact Male") + Q("sex_upon_outcome_Neutered Male") + Q("sex_upon_outcome_Spayed Female") + Q("sex_upon_outcome_Unknown") + Q("month_1") + Q("month_2") + Q("month_3") + Q("month_4") + Q("month_5") + Q("month_6") + Q("month_7") + Q("month_8") + Q("month_9") + Q("month_10") + Q("month_11") + Q("month_12")


In [29]:
Y, X = dmatrices(formula, df3, return_type='dataframe')

In [30]:
y = Y['target'].values
y[:10]

array([ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.])

In [31]:
model.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [32]:
print 'Prediction'
print model.predict(X[:10])
print 'Actual'
print y[:10]

Prediction
[ 0.  1.  1.  0.  0.  1.  0.  1.  1.  0.]
Actual
[ 0.  0.  1.  0.  0.  0.  1.  1.  0.  0.]


In [33]:
feature_importances = abs(model.feature_log_prob_[1] - model.feature_log_prob_[0])
feature_importances

array([ 0.10989251,  0.78658468,  0.12416694,  0.24789665,  0.55182773,
        2.20125766,  2.48752658,  0.50903803,  0.72406273,  7.01064725,
        0.14729117,  0.12442904,  0.17395665,  0.37837566,  0.29496977,
        0.02218546,  0.26084194,  0.13414308,  0.07010159,  0.10110577,
        0.00864699,  0.23260878])

In [34]:
feature_importance_series = Series(feature_importances, index=X.columns.values)
feature_importance_series[:50]

Q("age_binned_(-123.001, 78.0]")       0.109893
Q("age_binned_(78.0, 306.0]")          0.786585
Q("age_binned_(306.0, 669.0]")         0.124167
Q("age_binned_(669.0, 1222.0]")        0.247897
Q("age_binned_(1222.0, 9137.0]")       0.551828
Q("sex_upon_outcome_Intact Female")    2.201258
Q("sex_upon_outcome_Intact Male")      2.487527
Q("sex_upon_outcome_Neutered Male")    0.509038
Q("sex_upon_outcome_Spayed Female")    0.724063
Q("sex_upon_outcome_Unknown")          7.010647
Q("month_1")                           0.147291
Q("month_2")                           0.124429
Q("month_3")                           0.173957
Q("month_4")                           0.378376
Q("month_5")                           0.294970
Q("month_6")                           0.022185
Q("month_7")                           0.260842
Q("month_8")                           0.134143
Q("month_9")                           0.070102
Q("month_10")                          0.101106
Q("month_11")                          0

In [35]:
feature_importance_series.sort_values(ascending=False)[:50]

Q("sex_upon_outcome_Unknown")          7.010647
Q("sex_upon_outcome_Intact Male")      2.487527
Q("sex_upon_outcome_Intact Female")    2.201258
Q("age_binned_(78.0, 306.0]")          0.786585
Q("sex_upon_outcome_Spayed Female")    0.724063
Q("age_binned_(1222.0, 9137.0]")       0.551828
Q("sex_upon_outcome_Neutered Male")    0.509038
Q("month_4")                           0.378376
Q("month_5")                           0.294970
Q("month_7")                           0.260842
Q("age_binned_(669.0, 1222.0]")        0.247897
Q("month_12")                          0.232609
Q("month_3")                           0.173957
Q("month_1")                           0.147291
Q("month_8")                           0.134143
Q("month_2")                           0.124429
Q("age_binned_(306.0, 669.0]")         0.124167
Q("age_binned_(-123.001, 78.0]")       0.109893
Q("month_10")                          0.101106
Q("month_9")                           0.070102
Q("month_6")                           0

In [36]:
top_10_feature_indices = feature_importance_series.sort_values(ascending=False)[:50].index.values

In [37]:
inter_class_differences = model.feature_log_prob_[1] - model.feature_log_prob_[0]
new_feature_importance_series = Series(inter_class_differences, index=X.columns.values)

new_feature_importance_series[top_10_feature_indices]

Q("sex_upon_outcome_Unknown")         -7.010647
Q("sex_upon_outcome_Intact Male")     -2.487527
Q("sex_upon_outcome_Intact Female")   -2.201258
Q("age_binned_(78.0, 306.0]")          0.786585
Q("sex_upon_outcome_Spayed Female")    0.724063
Q("age_binned_(1222.0, 9137.0]")      -0.551828
Q("sex_upon_outcome_Neutered Male")    0.509038
Q("month_4")                          -0.378376
Q("month_5")                          -0.294970
Q("month_7")                           0.260842
Q("age_binned_(669.0, 1222.0]")       -0.247897
Q("month_12")                          0.232609
Q("month_3")                          -0.173957
Q("month_1")                           0.147291
Q("month_8")                           0.134143
Q("month_2")                           0.124429
Q("age_binned_(306.0, 669.0]")         0.124167
Q("age_binned_(-123.001, 78.0]")      -0.109893
Q("month_10")                         -0.101106
Q("month_9")                          -0.070102
Q("month_6")                          -0

In [38]:
df[df['animal_type'] == "Other"]["breed"].value_counts().head()

Bat Mix          1286
Bat               799
Raccoon Mix       465
Rabbit Sh Mix     291
Raccoon           230
Name: breed, dtype: int64

In [39]:
df[df['animal_type'] == "Dog"]["breed"].value_counts()[:100]

Pit Bull Mix                                6135
Chihuahua Shorthair Mix                     4733
Labrador Retriever Mix                      4608
German Shepherd Mix                         1892
Australian Cattle Dog Mix                   1059
Dachshund Mix                                798
Boxer Mix                                    674
Miniature Poodle Mix                         648
Border Collie Mix                            646
Catahoula Mix                                476
Rat Terrier Mix                              456
Australian Shepherd Mix                      454
Yorkshire Terrier Mix                        437
Siberian Husky Mix                           418
Jack Russell Terrier Mix                     416
Miniature Schnauzer Mix                      392
Beagle Mix                                   384
Chihuahua Longhair Mix                       359
Staffordshire Mix                            350
Great Pyrenees Mix                           330
Cairn Terrier Mix   

In [40]:
df['breed_count'] = df.groupby(['breed'])['animal_type'].transform('count')
df_breed = df[df['animal_type'] == "Dog"]
df_breed = df_breed[df_breed['breed_count'] >= 500]
df_breed

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,days_alive,has_name,target,age_binned,month,breed_count
7,3 years,A692618,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23,2014-12-08 15:55:00,2014-12-08T15:55:00,*Ella,Partner,Transfer,Spayed Female,1111,1,0.0,"(669.0, 1222.0]",12,4733
14,8 years,A690350,Dog,Labrador Retriever Mix,Black,2006-10-18,2014-10-26 18:20:00,2014-10-26T18:20:00,Shy,NaN,Return to Owner,Neutered Male,2930,1,0.0,"(1222.0, 9137.0]",10,4608
16,1 year,A674298,Dog,Pit Bull Mix,Brown Brindle/White,2013-03-11,2014-04-16 12:51:00,2014-04-16T12:51:00,*Newt,Partner,Transfer,Neutered Male,401,1,0.0,"(306.0, 669.0]",4,6135
27,2 months,A667311,Dog,Labrador Retriever Mix,Yellow,2013-09-01,2013-11-19 18:30:00,2013-11-19T18:30:00,Promise,NaN,Adoption,Spayed Female,79,1,1.0,"(78.0, 306.0]",11,4608
28,4 months,A690699,Dog,Chihuahua Shorthair Mix,Blue/Tan,2014-06-17,2014-11-04 18:03:00,2014-11-04T18:03:00,Minnie,Partner,Transfer,Intact Female,140,1,0.0,"(78.0, 306.0]",11,4733
32,3 months,A669918,Dog,Labrador Retriever Mix,Tan/White,2013-09-30,2014-01-03 17:47:00,2014-01-03T17:47:00,NaN,NaN,Adoption,Spayed Female,95,0,1.0,"(78.0, 306.0]",1,4608
34,12 years,A678098,Dog,Labrador Retriever Mix,Black/White,2002-05-08,2014-07-09 11:54:00,2014-07-09T11:54:00,Shades,Foster,Adoption,Neutered Male,4445,1,1.0,"(1222.0, 9137.0]",7,4608
36,2 years,A677038,Dog,Chihuahua Shorthair Mix,Buff,2012-04-22,2014-04-26 13:00:00,2014-04-26T13:00:00,*Penny,NaN,Adoption,Spayed Female,734,1,1.0,"(669.0, 1222.0]",4,4733
37,4 years,A665344,Dog,Chihuahua Shorthair Mix,Red/White,2009-10-16,2013-11-02 17:47:00,2013-11-02T17:47:00,Ginger,NaN,Adoption,Spayed Female,1478,1,1.0,"(1222.0, 9137.0]",11,4733
45,3 years,A647382,Dog,Border Collie Mix,Black/White,2010-08-09,2013-10-13 15:10:00,2013-10-13T15:10:00,Oreo,NaN,Adoption,Spayed Female,1161,1,1.0,"(669.0, 1222.0]",10,646


In [41]:
df_breed['color_count'] = df_breed.groupby(['color'])['animal_type'].transform('count')
df_breed_color = df_breed[df_breed['color_count'] >= 1000]
df_breed_color

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,days_alive,has_name,target,age_binned,month,breed_count,color_count
7,3 years,A692618,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23,2014-12-08 15:55:00,2014-12-08T15:55:00,*Ella,Partner,Transfer,Spayed Female,1111,1,0.0,"(669.0, 1222.0]",12,4733,1036
14,8 years,A690350,Dog,Labrador Retriever Mix,Black,2006-10-18,2014-10-26 18:20:00,2014-10-26T18:20:00,Shy,NaN,Return to Owner,Neutered Male,2930,1,0.0,"(1222.0, 9137.0]",10,4608,1517
32,3 months,A669918,Dog,Labrador Retriever Mix,Tan/White,2013-09-30,2014-01-03 17:47:00,2014-01-03T17:47:00,NaN,NaN,Adoption,Spayed Female,95,0,1.0,"(78.0, 306.0]",1,4608,1279
34,12 years,A678098,Dog,Labrador Retriever Mix,Black/White,2002-05-08,2014-07-09 11:54:00,2014-07-09T11:54:00,Shades,Foster,Adoption,Neutered Male,4445,1,1.0,"(1222.0, 9137.0]",7,4608,2775
45,3 years,A647382,Dog,Border Collie Mix,Black/White,2010-08-09,2013-10-13 15:10:00,2013-10-13T15:10:00,Oreo,NaN,Adoption,Spayed Female,1161,1,1.0,"(669.0, 1222.0]",10,646,2775
48,1 year,A693005,Dog,Boxer Mix,Brown,2013-11-29,2014-12-01 18:30:00,2014-12-01T18:30:00,Lex,NaN,Return to Owner,Neutered Male,367,1,0.0,"(306.0, 669.0]",12,674,1036
53,6 years,A545127,Dog,Pit Bull Mix,Black/White,2007-05-31,2014-05-04 15:20:00,2014-05-04T15:20:00,Ittybitty,Aggressive,Euthanasia,Spayed Female,2530,1,0.0,"(1222.0, 9137.0]",5,6135,2775
58,1 year,A677479,Dog,Chihuahua Shorthair Mix,Brown/White,2013-04-25,2014-05-01 17:05:00,2014-05-01T17:05:00,Tinker Bell,NaN,Adoption,Spayed Female,371,1,1.0,"(306.0, 669.0]",5,4733,1322
72,3 years,A674267,Dog,Chihuahua Shorthair Mix,Brown,2011-03-10,2014-03-18 16:04:00,2014-03-18T16:04:00,*Bambi,NaN,Adoption,Spayed Female,1104,1,1.0,"(669.0, 1222.0]",3,4733,1036
97,1 year,A688904,Dog,German Shepherd Mix,Tan/White,2013-09-26,2014-12-04 17:22:00,2014-12-04T17:22:00,*Flicker,Partner,Transfer,Spayed Female,434,1,0.0,"(306.0, 669.0]",12,1892,1279


In [42]:
df_breed_color['target'] = 0.0
mask = (df_breed_color['outcome_type'] == 'Adoption')
df_breed_color['target'][mask] = 1.0
df_breed_color['target'].value_counts()

0.0    4940
1.0    4067
Name: target, dtype: int64

In [43]:
categorical_columns = ['age_binned', 'sex_upon_outcome', 'month', 'breed']
df_dummies = pd.get_dummies(df[categorical_columns],
                            prefix=categorical_columns,
                            columns=categorical_columns)
dummy_column_names = df_dummies.columns.values
dummy_column_names[:10]

array(['age_binned_(-123.001, 78.0]', 'age_binned_(78.0, 306.0]',
       'age_binned_(306.0, 669.0]', 'age_binned_(669.0, 1222.0]',
       'age_binned_(1222.0, 9137.0]', 'sex_upon_outcome_Intact Female',
       'sex_upon_outcome_Intact Male', 'sex_upon_outcome_Neutered Male',
       'sex_upon_outcome_Spayed Female', 'sex_upon_outcome_Unknown'], dtype=object)

In [44]:
df4 = pd.concat([df_breed_color, df_dummies], axis=1)

In [45]:
formula = 'target ~ 0 + {}'.format(' + '.join(['Q("{}")'.format(x) for x in dummy_column_names]))
print formula

target ~ 0 + Q("age_binned_(-123.001, 78.0]") + Q("age_binned_(78.0, 306.0]") + Q("age_binned_(306.0, 669.0]") + Q("age_binned_(669.0, 1222.0]") + Q("age_binned_(1222.0, 9137.0]") + Q("sex_upon_outcome_Intact Female") + Q("sex_upon_outcome_Intact Male") + Q("sex_upon_outcome_Neutered Male") + Q("sex_upon_outcome_Spayed Female") + Q("sex_upon_outcome_Unknown") + Q("month_1") + Q("month_2") + Q("month_3") + Q("month_4") + Q("month_5") + Q("month_6") + Q("month_7") + Q("month_8") + Q("month_9") + Q("month_10") + Q("month_11") + Q("month_12") + Q("breed_Abyssinian Mix") + Q("breed_Affenpinscher Mix") + Q("breed_Afghan Hound Mix") + Q("breed_Afghan Hound/Labrador Retriever") + Q("breed_African Mix") + Q("breed_Airedale Terrier") + Q("breed_Airedale Terrier Mix") + Q("breed_Airedale Terrier/Irish Terrier") + Q("breed_Airedale Terrier/Labrador Retriever") + Q("breed_Airedale Terrier/Miniature Schnauzer") + Q("breed_Akbash Mix") + Q("breed_Akita") + Q("breed_Akita Mix") + Q("breed_Akita/Austra

In [46]:
Y, X = dmatrices(formula, df4, return_type='dataframe')

In [47]:
y = Y['target'].values
y[:10]

array([ 0.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  0.])

In [48]:
model.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [49]:
print 'Prediction'
print model.predict(X[:10])
print 'Actual'
print y[:10]

Prediction
[ 1.  0.  1.  0.  1.  1.  0.  1.  0.  1.]
Actual
[ 0.  0.  1.  1.  1.  0.  0.  1.  1.  0.]


In [50]:
prediction_train = model.predict(X)
print metrics.accuracy_score(y, prediction_train)

0.720883757078


In [51]:
feature_importances = abs(model.feature_log_prob_[1] - model.feature_log_prob_[0])
feature_importances

array([ 0.30734893,  0.54179   ,  0.33108624, ...,  0.17356906,
        0.17356906,  0.17356906])

In [52]:
feature_importance_series = Series(feature_importances, index=X.columns.values)
feature_importance_series[:20]

Q("age_binned_(-123.001, 78.0]")                                  0.307349
Q("age_binned_(78.0, 306.0]")                                     0.541790
Q("age_binned_(306.0, 669.0]")                                    0.331086
Q("age_binned_(669.0, 1222.0]")                                   0.250560
Q("age_binned_(1222.0, 9137.0]")                                  0.715927
Q("sex_upon_outcome_Intact Female")                               2.448056
Q("sex_upon_outcome_Intact Male")                                 2.624820
Q("sex_upon_outcome_Neutered Male")                               0.301855
Q("sex_upon_outcome_Spayed Female")                               0.543411
Q("sex_upon_outcome_Unknown")                                     4.170236
Q("month_1")                                                      0.043694
Q("month_2")                                                      0.246772
Q("month_3")                                                      0.182727
Q("month_4")             

In [53]:
feature_importance_series.sort_values(ascending=False)[:20]

Q("sex_upon_outcome_Unknown")                         4.170236
Q("sex_upon_outcome_Intact Male")                     2.624820
Q("sex_upon_outcome_Intact Female")                   2.448056
Q("age_binned_(1222.0, 9137.0]")                      0.715927
Q("breed_Australian Cattle Dog Mix")                  0.612482
Q("sex_upon_outcome_Spayed Female")                   0.543411
Q("age_binned_(78.0, 306.0]")                         0.541790
Q("breed_Pit Bull Mix")                               0.427551
Q("age_binned_(306.0, 669.0]")                        0.331086
Q("breed_Miniature Poodle Mix")                       0.327911
Q("age_binned_(-123.001, 78.0]")                      0.307349
Q("sex_upon_outcome_Neutered Male")                   0.301855
Q("month_8")                                          0.277110
Q("month_5")                                          0.253206
Q("age_binned_(669.0, 1222.0]")                       0.250560
Q("month_2")                                          0

In [54]:
top_10_feature_indices = feature_importance_series.sort_values(ascending=False)[:20].index.values

In [55]:
inter_class_differences = model.feature_log_prob_[1] - model.feature_log_prob_[0]
new_feature_importance_series = Series(inter_class_differences, index=X.columns.values)

new_feature_importance_series[top_10_feature_indices]

Q("sex_upon_outcome_Unknown")                        -4.170236
Q("sex_upon_outcome_Intact Male")                    -2.624820
Q("sex_upon_outcome_Intact Female")                  -2.448056
Q("age_binned_(1222.0, 9137.0]")                     -0.715927
Q("breed_Australian Cattle Dog Mix")                  0.612482
Q("sex_upon_outcome_Spayed Female")                   0.543411
Q("age_binned_(78.0, 306.0]")                         0.541790
Q("breed_Pit Bull Mix")                              -0.427551
Q("age_binned_(306.0, 669.0]")                        0.331086
Q("breed_Miniature Poodle Mix")                      -0.327911
Q("age_binned_(-123.001, 78.0]")                      0.307349
Q("sex_upon_outcome_Neutered Male")                   0.301855
Q("month_8")                                          0.277110
Q("month_5")                                         -0.253206
Q("age_binned_(669.0, 1222.0]")                      -0.250560
Q("month_2")                                          0